# この魚は何だろう

In [ ]:
import iris 
import io
import requests
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer

In [ ]:
from iris_config import IRIS_USER, IRIS_PASSWORD, IRIS_CONN_STR

In [ ]:

# LINEヤフーさんが提供している日本語対応のCLIPモデルを利用
HF_MODEL_PATH = 'line-corporation/clip-japanese-base'
device = "cuda" if torch.cuda.is_available() else "cpu"
# トークナイザー(テキストをモデル入力用のトークンに変換する役割)の読み込み
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
# 画像前処理（リサイズ・正規化など） を行うプロセッサーを読み込み
processor = AutoImageProcessor.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
# CLIP モデル本体を読み込み、指定したデバイス（CPU/GPU）に配置
model = AutoModel.from_pretrained(HF_MODEL_PATH, trust_remote_code=True).to(device)


In [ ]:
### 特徴ベクターを引数として、類似するものを取得
def get_similar_desc(conn, emb_str):
    try:
        sql = ("SELECT TOP 3 Name,VECTOR_COSINE(Features,TO_VECTOR(?,double)) FROM FishSearch.Fish fish "
        "ORDER BY VECTOR_COSINE(Features,TO_VECTOR(?,double)) DESC"
              )

        with conn.cursor() as cursor:
            cursor.execute(sql, [emb_str, emb_str])
            results = cursor.fetchall()
            retrieved_info = "\n".join(f"【魚の名前候補】 {row[0]} 【類似度スコア】 {row[1]}" for row in results)

        return retrieved_info

    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# 魚１ (スーパーで購入した魚)
image_path = "../datasets/super/1-IMG_20250712_175805466_AE3.png"

# 魚2 (スーパーで購入した魚)
#image_path="../datasets/super/2-IMG_20250712_175653584_B.jpg"

# 魚3（Wikipediaより取得 - 出典: サフィルによる著作物, "File:Hirame flatfish front.JPG", CC BY-SA 4.0)
# Link: https://commons.wikimedia.org/w/index.php?curid=45580662
#image_path = "../datasets/Wikipedia/image/ヒラメ.jpeg"

image = Image.open(image_path)
# イメージを表示
image.thumbnail((500, 300))
image

In [ ]:
# 画像をモデルの入力用テンソル形式に変換
image = processor(image, return_tensors="pt").to(device)

# 画像特徴量（ベクトル）を抽出
with torch.no_grad():
    image_features = model.get_image_features(**image)


In [ ]:
# ベクトルのフォーマット変換(Pytorchテンソル > カンマ区切りの文字列)
float_list = image_features[0].cpu().tolist()
emb_str = ",".join(map(str, float_list)) 

# IRISに接続
conn = iris.connect(IRIS_CONN_STR, IRIS_USER, IRIS_PASSWORD)

# 指定画像に類似するテキストを取得
info = get_similar_desc(conn, emb_str)
print(info)


# 
# 
# 

### 正解は・・

In [ ]:
# 魚1の正解
#image_path = "../datasets/super/truth/3-GroundTruth1_IMG_20250712_173303940_AE.jpg"

# 魚2の正解
image_path = "../datasets/super/truth/4-GroundTruth2IMG_20250712_173334354_AE.jpg"

image = Image.open(image_path)
# イメージを表示
image.thumbnail((700, 700))
image